In [1]:
import sys
sys.path.append('..')
import gdspy
import numpy as np
from importlib import reload
from copy import deepcopy
from QCreator import elements
from QCreator import general_sample_creator as creator
from QCreator import meshing
from QCreator.auxiliary_functions import *

tl_core = 20.
tl_gap = 12.
tl_ground = 10.


# resonator_core = 7
# resonator_gap = 4
# resonator_ground = 16
resonator_core = 8
resonator_gap = 7
resonator_ground = 16

pad_offset = 800
pad_element_offset = 1000
qubit_position_offset = 800

coupler_start = pad_offset + pad_element_offset
coupler_delta = 500
coupler_length = 450
num_couplers = 1

In [2]:
reload(gdspy)
reload(creator)
reload(elements)


layers_configuration = {
    'total':0,
    'restricted area':10,
    'for removing':100,
    'JJs':1,
    'air bridges':2,
    'air bridge pads':3,
    'vertical gridlines':15,
    'horizontal gridlines':16,
    'inverted':17
}

sample = creator.Sample('1Q_test',layers_configuration)

#specify sample vertical and horizontal lengths
sample.chip_geometry.sample_vertical_size=4.7e3
sample.chip_geometry.sample_horizontal_size=4.7e3
# num_qubits_line = 3

central_line_y = sample.chip_geometry.sample_vertical_size/2

# ChichkovAB = elements.AirBridge(width=15, 
#                                 length=60, 
#                                 padsize=30, 
#                                 min_spacing = 30,
#                                 layer_configuration=sample.layer_configuration)

chip_edge_ground = elements.ChipEdgeGround(sample.chip_geometry, sample.layer_configuration, sample.pads)
sample.add(chip_edge_ground)

In [3]:
# 1. Create contact pads:
pads_left = []
pads_right = []
for pad_side_id in range(1):
    pad = elements.Pad('pad-left-'+str(pad_side_id), 
                       (pad_offset,  sample.chip_geometry.sample_vertical_size/2), np.pi, tl_core, 
                       tl_gap, tl_ground, 
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                       **elements.default_pad_geometry())
    pads_left.append(pad)
    sample.add(pad)
    pad = elements.Pad('pad-right-'+str(pad_side_id), 
                       (sample.chip_geometry.sample_horizontal_size-pad_offset,  
                        sample.chip_geometry.sample_vertical_size/2), 0, tl_core, 
                       tl_gap, tl_ground, 
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                       **elements.default_pad_geometry())
    pads_right.append(pad)
    sample.add(pad)
    
pads_top = []
pads_bottom = []
for pad_side_id in range(3):
    pad = elements.Pad('pad-bottom-'+str(pad_side_id), 
                       (sample.chip_geometry.sample_horizontal_size/4*(pad_side_id+1), pad_offset), 
                       -np.pi/2, tl_core, tl_gap, tl_ground, 
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                       **elements.default_pad_geometry())
    pads_bottom.append(pad)
    sample.add(pad)
    pad = elements.Pad('pad-top-'+str(pad_side_id), 
                       (sample.chip_geometry.sample_horizontal_size/4*(pad_side_id+1),
                        sample.chip_geometry.sample_vertical_size-pad_offset), 
                        np.pi/2, tl_core, tl_gap, tl_ground, 
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                       **elements.default_pad_geometry())
    pads_top.append(pad)
    sample.add(pad)
    
p1 = pads_left[0]
p2 = pads_right[0]

In [4]:
# g5, g6, res, open_part_of_res = draw_single_resonator(sample,elements,
#                             coupler_start, central_line_y, coupler_length,
#                              resonator_core,resonator_gap,resonator_ground, 
#                              tl_core, tl_gap, tl_ground, grounding_width=10,
#                              closed_end_meander_length=3000,length_left=200,length_right=400,
#                              open_end_legth=800,
#                              port_orientation='right', direction_orientation='down')

In [5]:
connector = sample.connect_cpw(o1=p1, o2=p2, port1='port', port2='port', name='---',
                    points=[], bridges = True, pads_geometry=[4,10],
                    bridge_geometry=[56, 7], distance_between_pads=54, min_spacing=300)

cpw segments [{'type': 'endpoint', 'endpoint': array([ 800., 2350.])}, {'type': 'segment', 'endpoint': array([ 800.19200006, 2350.        ]), 'startpoint': array([ 800., 2350.])}, {'type': 'segment', 'endpoint': array([3899.80799994, 2350.        ]), 'startpoint': array([ 800.19200006, 2350.        ])}, {'type': 'endpoint', 'endpoint': array([3900., 2350.]), 'startpoint': array([3899.80799994, 2350.        ])}]
[CPW "---0", l=0.192, CPW "---_between_bridges_1", l=300.000, CPW "---_between_bridges_2", l=300.000, CPW "---_between_bridges_3", l=300.000, CPW "---_between_bridges_4", l=300.000, CPW "---_between_bridges_5", l=300.000, CPW "---_between_bridges_6", l=300.000, CPW "---_between_bridges_7", l=300.000, CPW "---_between_bridges_8", l=300.000, CPW "---_between_bridges_9", l=300.000, CPW "---_between_bridges_10", l=309.616]


In [6]:
connector.g

10.0

In [17]:
sample.draw_design()
sample.watch()

In [23]:
sample.write_to_gds()

Gds file has been writen here:  /home/alena/QCreator/example_notebooks\1Q_test.gds


In [6]:
sys, connections, elements = sample.get_tls()

In [7]:
sample.get_topology()

[[Short , [0]],
 [Port pad-left-0, [20]],
 [Port pad-right-0, [21]],
 [Port pad-bottom-0, [22]],
 [Port pad-top-0, [23]],
 [Port pad-bottom-1, [24]],
 [Port pad-top-1, [25]],
 [Port pad-bottom-2, [26]],
 [Port pad-top-2, [27]],
 [TL ---0 (n=1), [20, 1]],
 [TL ---_between_bridges_1 (n=1), [1, 2]],
 [TL ---_between_bridges_2 (n=1), [3, 4]],
 [TL ---_between_bridges_3 (n=1), [5, 6]],
 [TL ---_between_bridges_4 (n=1), [7, 8]],
 [TL ---_between_bridges_5 (n=1), [9, 10]],
 [TL ---_between_bridges_6 (n=1), [11, 12]],
 [TL ---_between_bridges_7 (n=1), [13, 14]],
 [TL ---_between_bridges_8 (n=1), [15, 16]],
 [TL ---_between_bridges_9 (n=1), [17, 18]],
 [TL ---_between_bridges_10 (n=1), [19, 21]],
 [TL ---bridge0_line1 (n=1), [2, 28]],
 [TL ---bridge0_line2 (n=1), [28, 3]],
 [C , [28, 0]],
 [TL ---bridge1_line1 (n=1), [4, 29]],
 [TL ---bridge1_line2 (n=1), [29, 5]],
 [C , [29, 0]],
 [TL ---bridge2_line1 (n=1), [6, 30]],
 [TL ---bridge2_line2 (n=1), [30, 7]],
 [C , [30, 0]],
 [TL ---bridge3_line1

In [8]:
omega, kappa, modes = sys.get_modes()

In [9]:
omega / (2* np.pi)

array([1.70537917e+10, 3.47195627e+10, 5.31294065e+10, 7.21126031e+10,
       9.15056097e+10, 1.11244927e+11, 1.31383320e+11, 1.52184508e+11,
       1.74838918e+11, 2.06030064e+11, 2.24902078e+11, 2.31578616e+11,
       2.31850668e+11, 2.55716845e+11, 2.77907954e+11, 2.99266820e+11,
       3.20435317e+11, 3.41825905e+11, 3.64010368e+11, 3.99969948e+11,
       4.23042851e+11, 4.23076281e+11, 4.36102079e+11, 4.45133723e+11,
       4.65215214e+11, 4.82713897e+11, 4.97650108e+11, 5.10375395e+11,
       5.21277615e+11, 5.37073994e+11, 5.38230058e+11, 5.41084265e+11,
       5.46446119e+11, 5.53716482e+11, 5.54133376e+11, 5.58159835e+11,
       5.61556695e+11, 5.69040592e+11, 5.75323587e+11, 5.76954424e+11,
       5.85762870e+11, 5.93312915e+11, 5.95949671e+11, 6.02327913e+11,
       6.19025928e+11, 6.19428556e+11, 6.20561731e+11, 6.21716239e+11,
       6.22591989e+11, 6.22952938e+11, 6.28091695e+11, 6.28475522e+11,
       6.32410832e+11, 6.32694768e+11, 6.33683571e+11, 6.34469770e+11,
      

In [ ]:
1.930694